Note: DenseNet currently doesn't work with LIME due to a conflict between DenseNet's preprocessing function and LIME's dependency on scikit-image

In [ ]:
import os
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Lambda, Input

from skimage.color import gray2rgb, rgb2gray 
from skimage.util.montage import montage2d

print('Notebook run using keras:', keras.__version__)

In [ ]:
# define metric
from balancedAccuracy import balancedAccuracy
num_classes = 3
bacc_metric = balancedAccuracy(num_classes)

In [ ]:
model = keras.models.load_model(".../inceptionresnetv2.h5",
                                custom_objects={"balanced_acc":bacc_metric.balanced_acc})

In [ ]:
model.summary()

In [ ]:
testdata = np.load(".../TESTHAM.npz")
testList = testdata["testList"]
targetTestList = testdata["targetTestList"]

In [ ]:
from keras.applications.inception_resnet_v2 import preprocess_input
testListProc = preprocess_input(testList[0])

In [ ]:
np.where(np.argmax(targetTestList, 1)==2)

In [ ]:
choiceSample = 0  # change this

In [ ]:
targetTestList[choiceSample] 

In [ ]:
preds = model.predict(testListProc[0].reshape(1,224,224,3))

In [ ]:
%load_ext autoreload
%autoreload 2
import os,sys
import lime

In [ ]:
from lime.wrappers.scikit_image import SegmentationAlgorithm
from lime import lime_image
explainer = lime_image.LimeImageExplainer(verbose = False)
segmenter = SegmentationAlgorithm('slic', n_segments=500, compactness=1, sigma=1)

In [ ]:
%%time
explanation = explainer.explain_instance(testListProc[choiceSample], 
                                         classifier_fn = model.predict, 
                                         top_labels=3, num_samples=500, segmentation_fn=segmenter)

In [ ]:
from skimage.color import label2rgb
from skimage.segmentation import mark_boundaries

temp, mask = explanation.get_image_and_mask(np.argmax(targetTestList[choiceSample]), positive_only=False,
                                            num_features=10, hide_rest=False)
fig, (ax1) = plt.subplots(1,1, figsize = (8, 4))

ax1.axis("off")
ax1.imshow(mark_boundaries(temp / 2 + 0.5, mask))


np.set_printoptions(suppress=True)
np.set_printoptions(precision=3)
ax1.set_title(['{:.3f}'.format(i) for i in preds[choiceSample]])
ax1.title.set_fontsize(18)


plt.savefig("....png", dpi=500)